In [1]:
# !pip install tensorflow==2.9.2 tensorflow-gpu==2.9.2 opencv-python mediapipe matplotlib

In [2]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import os

In [3]:
mp_holistic=mp.solutions.holistic  #Holistic Model
mp_drawing=mp.solutions.drawing_utils  #Drawing Utilities

In [4]:
def mediapipe_detection(image,model):
    image=cv.cvtColor(image,cv.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)    #Making Pridictions
    image.flags.writeable=True
    image=cv.cvtColor(image,cv.COLOR_RGB2BGR)
    return image,results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)   # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)       # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)  # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(160,32,240), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(240,32,160), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(161,17,13), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(161,17,13), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(161,17,13), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2)
                             )

In [7]:
cap=cv.VideoCapture(0)
# To access mediapipe model:
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image,results=mediapipe_detection(frame,holistic)
        draw_styled_landmarks(image,results)
        cv.imshow('Feed',image)
        if cv.waitKey(10) & 0xFF==ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

## Extracting Keypoint Values

In [6]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

NameError: name 'results' is not defined

In [7]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

## Folders for Collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = 'MP_Data'
# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])
# Thirty videos worth of data
no_sequences = 30
# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
# Will create 4 folders
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

## Collecting Data for Training and Testing

In [12]:
# cap = cv.VideoCapture(0)
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     for action in actions:
#         # Loop through sequences
#         for sequence in range(no_sequences):
#             # Loop through video length
#             for frame_num in range(sequence_length):
#                 ret, frame = cap.read()
#                 # Make detections
#                 image, results = mediapipe_detection(frame, holistic)
#                 draw_styled_landmarks(image, results)
#                 if frame_num == 0: 
#                     cv.putText(image, 'STARTING COLLECTION', (120,200), 
#                                cv.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv.LINE_AA)
#                     cv.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv.LINE_AA)
#                     cv.imshow('OpenCV Feed', image)
#                     cv.waitKey(2000)
#                 else: 
#                     cv.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
#                                cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv.LINE_AA)
#                     cv.imshow('OpenCV Feed', image)
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)
#                 if cv.waitKey(10) & 0xFF == ord('q'):
#                     break
#     cap.release()
#     cv.destroyAllWindows()

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

KeyboardInterrupt: 

In [11]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

NameError: name 'sequences' is not defined

# Building and Training LSTM Neural Network

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [13]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [14]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
# model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [19]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
3/3 [==============================] - 3s 179ms/step - loss: 1.8937 - categorical_accuracy: 0.3412
Epoch 2/200
3/3 [==============================] - 0s 157ms/step - loss: 1.4579 - categorical_accuracy: 0.3412
Epoch 3/200
3/3 [==============================] - 1s 177ms/step - loss: 1.0985 - categorical_accuracy: 0.4824
Epoch 4/200
3/3 [==============================] - 1s 176ms/step - loss: 1.3606 - categorical_accuracy: 0.3412
Epoch 5/200
3/3 [==============================] - 1s 182ms/step - loss: 1.0036 - categorical_accuracy: 0.5294
Epoch 6/200
3/3 [==============================] - 1s 183ms/step - loss: 1.0121 - categorical_accuracy: 0.6118
Epoch 7/200
3/3 [==============================] - 1s 186ms/step - loss: 1.4653 - categorical_accuracy: 0.4706
Epoch 8/200
3/3 [==============================] - 1s 196ms/step - loss: 1.2773 - categorical_accuracy: 0.2824
Epoch 9/200
3/3 [==============================] - 0s 164ms/step - loss: 1.0857 - categorical_accuracy: 0.4000
E

KeyboardInterrupt: 

# Saving Weights

In [20]:
model.save('action2.h5')

In [40]:
del model

In [15]:
model.load_weights('actionOther.h5')

# Real Time Testing

In [16]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv.putText(output_frame, actions[num], (0, 85+num*40), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv.LINE_AA)
    return output_frame

In [25]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
            if len(sentence) > 5: 
                sentence = sentence[-5:]
            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
        cv.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv.putText(image, ' '.join(sentence), (3,30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
        # Show to screen
        cv.imshow('OpenCV Feed', image)
        # Break gracefully
        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

hello   [9.7280079e-01 6.0001448e-06 2.7193213e-02]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 44ms/step
hello   [9.4683278e-01 9.2367891e-06 5.3157996e-02]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 33ms/step
hello   [8.9923626e-01 1.3445728e-05 1.0075031e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
hello   [8.3115500e-01 1.7262948e-05 1.6882776e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 46ms/step
hello   [7.2749019e-01 2.1447506e-05 2.7248836e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 40ms/step
hello   [6.1956698e-01 2.4885285e-05 3.8040814e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 51ms/step
hello   [5.3478688e-01 2.7254131e-

1/1 [==============================] - 0s 42ms/step
iloveyou   [2.4504586e-01 3.9366365e-05 7.5491476e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 52ms/step
iloveyou   [1.5038562e-01 2.5410949e-05 8.4958899e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
iloveyou   [9.3955040e-02 1.5939708e-05 9.0602899e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 32ms/step
iloveyou   [6.4268708e-02 1.0140155e-05 9.3572116e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 52ms/step
iloveyou   [4.8438501e-02 6.5476447e-06 9.5155495e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 70ms/step
iloveyou   [3.8151566e-02 4.4960771e-06 9.6184391e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [===========

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 51ms/step
hello   [9.9999845e-01 1.7283124e-09 1.5004196e-06]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 53ms/step
hello   [9.9999833e-01 1.9078019e-09 1.6561044e-06]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 54ms/step
hello   [9.9999332e-01 3.1400349e-09 6.7124934e-06]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 53ms/step
hello   [9.9996042e-01 1.0222530e-08 3.9518374e-05]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 48ms/step
hello   [9.9986410e-01 1.6502772e-08 1.3592951e-04]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
hello   [9.9902868e-01 4.5974858e-08 9.7131793e-04]
<class 'mediapipe.python.solution_

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 47ms/step
iloveyou   [4.1995370e-03 1.6805195e-08 9.9580044e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 42ms/step
iloveyou   [3.8148023e-03 4.8361732e-09 9.9618524e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 68ms/step
iloveyou   [5.4375310e-03 7.0542239e-09 9.9456245e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 33ms/step
iloveyou   [6.9355308e-03 1.0196921e-08 9.9306446e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 47ms/step
iloveyou   [1.5825519e-02 6.1260863e-08 9.8417443e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 43ms/step
iloveyou   [6.9329411e-02 6.6956125e-07 9.3066990e-01]
<class 'mediapip

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 49ms/step
iloveyou   [8.8198138e-03 1.8305976e-06 9.9117833e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 41ms/step
iloveyou   [3.7253650e-03 4.8360007e-06 9.9626982e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 66ms/step
iloveyou   [1.8198164e-03 2.2995191e-05 9.9815720e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 26ms/step
iloveyou   [1.5618147e-03 8.3094137e-04 9.9760723e-01]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 62ms/step
iloveyou   [0.0015218  0.06207543 0.9364028 ]
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 61ms/step
thanks   [5.5333669e-04 8.0348748e-01 1.9595912e-01]
<class 'mediapipe.python.so